In [2]:
import numpy as np
import scipy as sp
import sklearn as sk
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pylab as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn as sns
import statsmodels as sm
import statsmodels.api as sm
import pandas as pd
from patsy import dmatrix

C:\Users\ledes\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
from geopy.geocoders import Nominatim

In [4]:
data = pd.read_csv("train.csv")

In [6]:
from geopy.geocoders import Nominatim
from urllib.request import Request

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

location = get_geolocator().reverse#(lat, long)
address = location.address

AttributeError: 'function' object has no attribute 'address'

In [7]:
pickup_lat = data["pickup_latitude"]
pickup_lon = data["pickup_longitude"]    

In [39]:
post_code = pd.DataFrame(columns = ["post_code"])
post_code2 = pd.DataFrame(columns = ["post_code"])

2
1


In [30]:
import threading

In [37]:
def reverse_geocode() :
    for a, b in list(zip(pickup_lat, pickup_lon))[ : 700000] :
        location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 3)
        address = location.address
        print("1")
        post_code.loc[len(post_code)] = address[-31 : -26]

def reverse_geocode2() :
    for a, b in list(zip(pickup_lat, pickup_lon))[700000 : ] :
        location = get_geolocator().reverse(str(a) + ',' + str(b), timeout = 3)
        address = location.address
        print("2")
        post_code2.loc[len(post_code)] = address[-31 : -26]

In [40]:
th1 = threading.Thread(target = reverse_geocode)
th1.start()

th2 = threading.Thread(target = reverse_geocode2)
th2.start()

2
1
1
2
2
1
1
2
2
1
1
2
2
1
1
2
2
1
2
1
2
1
1
2
2
1
2
1
2
1
1
2
1
2
2
1
2
1
1
2
2
1
1
2
2
1
1
2
2
1
1
2
2
1
1
2
2
1
1
2
1
1
2
2
1
1
2
2
1
1
2
2
1
2
2
1
1
1
2
1
1
2
2
1
2
2
1
1
2
1
2
2
1
1
2
2
1
1
2
2
1
1
2
1
2
1
2
2
1
2
2
1
2
1
2
1
1
2
1
2
1
2
1
2
1
2
2
1
1
2
1
1
2
2
1
2
1
2
2
1
2
2


Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\ledes\Anaconda3\lib\site-packages\geopy\geocoders\base.py", line 143, in _call_geocoder
    page = requester(req, timeout=(timeout or self.timeout), **kwargs)
  File "<ipython-input-6-2fca22ce2302>", line 11, in requester_hack
    return requester(req, **kwargs)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1321, in do_o

1
2
2
1
2
2
1
2
2
1
2
1
2
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
2
2
1
1
2
1
2


Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\ledes\Anaconda3\lib\site-packages\geopy\geocoders\base.py", line 143, in _call_geocoder
    page = requester(req, timeout=(timeout or self.timeout), **kwargs)
  File "<ipython-input-6-2fca22ce2302>", line 11, in requester_hack
    return requester(req, **kwargs)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1321, in do_

2
1
1
2
2
1
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
1
2
1
2
1
2
1
2
1
2
1
2
2
1
2
1
2
1
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
2
1
2
1
1
2
1
2
1
2
1
2
1
2
1
2
1
2
1
2
2
1
1
1


Exception in thread Thread-9:
Traceback (most recent call last):
  File "C:\Users\ledes\Anaconda3\lib\site-packages\geopy\geocoders\base.py", line 143, in _call_geocoder
    page = requester(req, timeout=(timeout or self.timeout), **kwargs)
  File "<ipython-input-6-2fca22ce2302>", line 11, in requester_hack
    return requester(req, **kwargs)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1321, in do_o

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\ledes\Anaconda3\lib\site-packages\geopy\geocoders\base.py", line 143, in _call_geocoder
    page = requester(req, timeout=(timeout or self.timeout), **kwargs)
  File "<ipython-input-6-2fca22ce2302>", line 11, in requester_hack
    return requester(req, **kwargs)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 223, in urlopen
    return opener.open(url, data, timeout)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 526, in open
    response = self._open(req, data)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 544, in _open
    '_open', req)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 504, in _call_chain
    result = func(*args)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1361, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\ledes\Anaconda3\lib\urllib\request.py", line 1321, in do_

In [43]:
len(post_code)
    

169

In [44]:
len(post_code2)


110